<a href="https://colab.research.google.com/github/Jack3690/INSIST_pipeline/blob/main/notebooks/MACADAMIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install insist-pista --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 57.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.0/857.0 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 kB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 89.6 MB/s eta 0:00:00


In [16]:
!pip install astroquery --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 75.2 MB/s eta 0:00:00


In [18]:
from astroquery.astrometry_net import AstrometryNet

In [2]:
import pista as pt
import numpy as np
import os
import matplotlib.pyplot as plt

import matplotlib.colors as col
from astropy.io import fits
from astropy.table import Table

In [3]:
data_path = pt.data_dir

In [4]:
!git clone https://github.com/Jack3690/INSIST_pipeline --quiet

# **Image Generation using PISTA**

In [6]:
if not os.path.exists('input_data'):
  os.mkdir('input_data')

In [7]:
# Example using FITS table
tab = Table.read(f'{data_path}/sample.fits')  # FITS Table
df  = tab.to_pandas()             # PISTA requires pandas DataFrame

In [9]:
tel_params = {
              'aperture'       : 100,
              'pixel_scale'    : 0.1,
              'psf_file'       : f'{data_path}/PSF/INSIST/off_axis_hcipy.npy',
              'response_funcs' :  [ f'{data_path}/INSIST/UV/Filter.dat,1,100',
                                    f'{data_path}/INSIST/UV/Coating.dat,5,100',   # 5 mirrors
                                    f'{data_path}/INSIST/UV/Dichroic.dat,2,100',   # 2 dichroics
                                  ],
              }

det_params = {    'shot_noise' :  'Gaussian',
                  'G1'         :  1,
                  'PRNU_frac'  :  0.25/100,
                  'qe_response': [f'{data_path}/INSIST/UV/QE.dat,1,100'],
                  'RN'         :  5,
                  'T'          :  218,
                  'DCNU'       :  0.1/100
             }

In [10]:
sim = pt.Imager(df = df,tel_params = tel_params, n_x = 500, n_y = 500,
                exp_time = 2400)

user_profiles = {'PRNU' : sim.flat_frame,
                 'Bias' : sim.bias_array,
                 'DNFP' : sim.DNFP_array}

sim(det_params = det_params, photometry = None)

for i, t in enumerate([800, 600, 1000, 1000, 600]):

  del_ra = np.random.random(1)[0]/3600
  del_dec = np.random.random(1)[0]/3600

  coords = (11.0689964234 + del_ra, 41.3520244153 + del_dec)

  sim = pt.Imager(df = df, coords = coords,
                  tel_params = tel_params, n_x = 500, n_y = 500,
                  exp_time = t, user_profiles = user_profiles)
  sim( det_params = det_params, photometry = None)

  sim.writeto(f'input_data/img{i}.fits')
  flux = 10**(0.4*sim.ZP)/sim.exp_time
  ZP = 2.5*np.log10(flux)

# **Pipeline**

## **Calibration**

In [34]:
input_dir = '/content/input_data'
cal_db = '/content/INSIST_pipeline/data/Cal_DB'

dark = fits.open(f'{cal_db}/Master_Dark.fits')[0].data
bias = fits.open(f'{cal_db}/Master_Bias.fits')[0].data
flat = fits.open(f'{cal_db}/Master_Flat.fits')[0].data

out_dir = 'Out_FC'
if not os.path.exists(out_dir):
  os.mkdir(out_dir)
for f in os.listdir(input_dir):

  hdul = fits.open(f'{input_dir}/{f}')
  data = hdul[0].data/hdul[0].header['EXPTIME']

  # Dark and Bias Subtraction
  data_DB = data - bias - dark

  # Flat fielding
  data_F = data_DB/flat

  hdul[0].data = data_F
  hdul[0].header['FRAME'] = "FLUX CALIBRATED"
  hdul[0].header['BUNIT'] ='ADU'
  hdul[0].header['ZP'] = 22.239
  hdul.writeto(f"{out_dir}/{f.split('.')[0]}_FC.fits", overwrite = True)

## **Pre-processing**

In [ ]:
%history

## **Astrometric Calibration**

In [38]:
out_dir = 'Out_FC'

ast = AstrometryNet()
ast.api_key = 'pmkaisuqszgpqvls'
for f in os.listdir(out_dir):
  dir = f'{out_dir}/{f}'
  out = ast.solve_from_image(dir,ra_key='CRVAL1',
                       dec_key='CRVAL2',
                       ra_dec_units = 'degree',
                       solve_timeout=30)
  break


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Determining background stats
Finding sources
Found 14 sources
 id     xcentroid      ...        flux                mag         
--- ------------------ ... ------------------ --------------------
  6  278.0627118823046 ...  4.620996678883115  -1.6618391409768885
  7 288.06674564937026 ...   4.58960064274397  -1.6544372442363253
 12  71.94982972212146 ...  3.804860045048083  -1.4508467166421748
 10  280.0718413064401 ...  3.773681160802485  -1.4419130093433359
  2 352.97093198891105 ... 3.4868073891396523  -1.3560698941467344
  4 268.01404014069163 ... 2.7657274951797786  -1.1045234680215745
  5   291.057062419228 ... 2.0494797963805143  -0.7791091035772574
  8 126.14697495891298 ... 1.9842497301293378  -0.7439908247771208
 13 418.02858517002915 ... 1.5545281462498721 -0.47899647444005533
 11 282.06741491144044 ... 1.4953806530350546 -0.43687939380849816
 14 168.07448295

In [39]:
out

{}